In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    print("Mounting Failed.")

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/DL_Project/

/content/drive/MyDrive/DL_Project


In [33]:
import torch
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import imageio
import os


In [34]:
from model import datasets

In [35]:
# CLASS_IDS_ALL = (
#     '02691156,02828884,02933112,02958343,03001627,03211117,03636649,' +
#     '03691459,04090263,04256520,04379243,04401088,04530566')

CLASS_IDS_ALL = (
    '02691156')

DATA_PATH = "./data/datasets"

In [31]:
test_dataset = datasets.ShapeNet(2, DATA_PATH, CLASS_IDS_ALL.split(','), 'test')

test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

Loading dataset: 100%|██████████| 1/1 [00:01<00:00,  1.49s/it]


In [36]:
def voxel2mesh(voxels, surface_view):
    # taken from https://github.com/chrischoy/3D-R2N2/blob/master/lib/voxel.py
    cube_verts = [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 0, 1], [1, 1, 0],
                  [1, 1, 1]]  # 8 points

    cube_faces = [[0, 1, 2], [1, 3, 2], [2, 3, 6], [3, 7, 6], [0, 2, 6], [0, 6, 4], [0, 5, 1],
                  [0, 4, 5], [6, 7, 5], [6, 5, 4], [1, 7, 3], [1, 5, 7]]  # 12 face

    cube_verts = np.array(cube_verts)
    cube_faces = np.array(cube_faces) + 1

    scale = 0.01
    cube_dist_scale = 1.1
    verts = []
    faces = []
    curr_vert = 0
    
    voxels = voxels.cpu().numpy()
    
    positions = np.where(voxels > 0.3)
    voxels[positions] = 1 
    for i, j, k in zip(*positions):
        # identifies if current voxel has an exposed face 
        if not surface_view or np.sum(voxels[i-1:i+2, j-1:j+2, k-1:k+2]) < 27:
            verts.extend(scale * (cube_verts + cube_dist_scale * np.array([[i, j, k]])))
            faces.extend(cube_faces + curr_vert)
            curr_vert += len(cube_verts)  
              
    return np.array(verts), np.array(faces)
    
    
def write_obj(filename, verts, faces):
    # taken from https://github.com/chrischoy/3D-R2N2/blob/master/lib/voxel.py:
    """ write the verts and faces on file."""
    with open(filename, 'w') as f:
        # write vertices
        f.write('g\n# %d vertex\n' % len(verts))
        for vert in verts:
            f.write('v %f %f %f\n' % tuple(vert))

        # write faces
        f.write('# %d faces\n' % len(faces))
        for face in faces:
            f.write('f %d %d %d\n' % tuple(face))

    
def voxel2obj(filename, pred, surface_view = True):
    # taken from https://github.com/chrischoy/3D-R2N2/blob/master/lib/voxel.py
    verts, faces = voxel2mesh(pred, surface_view)
    write_obj(filename, verts, faces)

In [37]:
for batch, data in enumerate(test_dataloader):
    if batch == 5:
      images = torch.autograd.Variable(data["img"][:,0,:,:,:])
      viewpoints = data["view"][:,0,:]
      # voxels = data["vox"].numpy()

      # print(voxels.shape)
      # print(images.shape)
      # print(viewpoints)
      img = (255. * images[0]).detach().cpu().numpy().clip(0, 255).astype('uint8').transpose(1, 2, 0)
      path = 'visual_input1.png'
      imageio.imsave(path, img)

      v = 1
      images_ = torch.autograd.Variable(data["img"][:,v,:,:,:])
      viewpoints_ = data["view"][:,v,:]
      # print(images_[0][0])
      # print(viewpoints_)

      img = (255. * images_[0]).detach().cpu().numpy().clip(0, 255).astype('uint8').transpose(1, 2, 0)
      path = 'visual_input2.png'
      imageio.imsave(path, img)

      path = 'truth.obj'
      voxel2obj(path, data["vox"][0,:,:,:])

            